In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 1. Load your data (already preprocessed and scaled)
df = pd.read_csv("telco_customer_preprocessed.csv")

# 2. Preprocess: Select features and target
features = [
    'tenure', 'MonthlyCharges_scaled', 'TotalCharges_scaled', 'gender_Male', 'gender_Female',
    'OnlineBackup_FE', 'InternetService_FE', 'OnlineSecurity_FE', 'TechSupport_FE', 'Contract_FE',
    'PaymentMethod_FE', 'PaperlessBilling_FE', 'MultipleLines_FE', 'DeviceProtection_FE',
    'Partner_Yes', 'Partner_No', 'SeniorCitizen', 'Dependents_No', 'Dependents_Yes',
    'StreamingTV_FE', 'StreamingMovies_FE'
]
target = 'Churn'  # 0 or 1

# Convert categorical columns if needed (example)
if df['Churn'].dtype == object:
    df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

# Check for missing columns and missing values
missing_cols = [col for col in features + [target] if col not in df.columns]
if missing_cols:
    raise ValueError(f"Missing columns in data: {missing_cols}")

df = df.dropna(subset=features + [target])
if df.shape[0] == 0:
    raise ValueError("No data left after dropping rows with missing values. Check your data and preprocessing.")

X = df[features].values.astype(np.float32)
y = df[target].values.astype(np.float32)

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Convert to PyTorch tensors (no scaling here)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# 5. Define ANN model
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()  # For binary classification
        )
    def forward(self, x):
        return self.net(x)

model = ANN(input_dim=X_train.shape[1])

# 6. Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 8. Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_label = (y_pred > 0.5).float()
    accuracy = (y_pred_label.eq(y_test_tensor).sum() / float(y_test_tensor.shape[0])).item()
    print(f"Test Accuracy: {accuracy:.4f}")

# 9. Inspect weights (for research)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

Epoch [10/50], Loss: 0.7673
Epoch [20/50], Loss: 0.7068
Epoch [30/50], Loss: 0.6358
Epoch [40/50], Loss: 0.5643
Epoch [50/50], Loss: 0.5245
Test Accuracy: 0.7353
net.0.weight tensor([[ 9.9997e-02,  8.2125e-02,  1.2853e-01, -1.8056e-01, -1.3870e-03,
         -1.8595e-01,  8.0663e-02,  7.9543e-02, -1.2015e-02, -7.1955e-02,
          1.5648e-01, -7.7411e-02, -2.3129e-01,  1.4056e-01, -1.3999e-01,
         -8.0583e-02, -1.1184e-01, -3.5933e-02, -1.2522e-01,  1.6936e-01,
         -9.5907e-02],
        [-2.5344e-02,  6.9210e-02,  2.0009e-02,  2.4225e-01, -1.2113e-01,
         -2.1556e-03,  6.6050e-02,  2.7223e-01,  4.4465e-02, -5.7258e-02,
          2.4913e-02, -1.5667e-01,  1.1302e-01, -1.4411e-01,  1.8568e-01,
         -2.5268e-02,  4.1748e-02,  1.0102e-01,  6.8141e-02,  2.5159e-01,
         -1.4037e-01],
        [-1.4082e-01, -1.4786e-01, -6.3041e-02,  2.3603e-01, -1.2970e-02,
         -1.5465e-01, -8.6819e-02, -1.3962e-01, -7.3415e-02,  9.8984e-02,
         -7.7937e-02,  5.8981e-02,  7.2